In [16]:
def Hb(p1, q1, r1):
    return 1 + 2*p1*q1 + 6*p1*q1*r1 + 2*p1*r1 + 2*q1*r1 - p1 - q1 - r1

row_format ="{:>5}" * 4
print(row_format.format('p1', 'q1', 'r1', 'E'))
for i in range(8):
    x = [(i >> j) & 1 for j in range(3)]
    E = Hb(x[0], x[1], x[2])
    print(row_format.format(*x,E))
    
    

   p1   q1   r1    E
    0    0    0    1
    1    0    0    0
    0    1    0    0
    1    1    0    1
    0    0    1    0
    1    0    1    1
    0    1    1    1
    1    1    1   10


In [18]:
def Hqb(p1, q1, r1, s1):
    return -6*p1*s1 - 6*q1*s1 - 6*r1*s1 + 8*p1*q1 + 8*p1*r1 + 8*q1*r1 + 6*s1 - p1 - q1 - r1 + 1

row_format ="{:>5}" * 5
print(row_format.format('p1', 'q1', 'r1', 's1', 'E'))
for i in range(16):
    x = [(i >> j) & 1 for j in range(4)]
    E = Hqb(x[0], x[1], x[2], x[3])
    print(row_format.format(*x,E))

   p1   q1   r1   s1    E
    0    0    0    0    1
    1    0    0    0    0
    0    1    0    0    0
    1    1    0    0    7
    0    0    1    0    0
    1    0    1    0    7
    0    1    1    0    7
    1    1    1    0   22
    0    0    0    1    7
    1    0    0    1    0
    0    1    0    1    0
    1    1    0    1    1
    0    0    1    1    0
    1    0    1    1    1
    0    1    1    1    1
    1    1    1    1   10


In [21]:
def Hs(s1, s2, s3, s4):
     return -1.5*s1*s4 - 1.5*s2*s4 - 1.5*s3*s4 + 2*s1*s2 + 2*s1*s3 + 2*s2*s3 - 3*s1 - 3*s2 - 3*s3 + 7.5*s4 + 11
    
row_format ="{:>6}" * 5
print(row_format.format('s1', 's2', 's3', 's4', 'E'))
for i in range(16):
    x = [1 - 2*((i >> j) & 1) for j in range(4)]
    E = Hs(x[0], x[1], x[2], x[3])
    print(row_format.format(*x,E))

    s1    s2    s3    s4     E
     1     1     1     1  11.0
    -1     1     1     1  12.0
     1    -1     1     1  12.0
    -1    -1     1     1  21.0
     1     1    -1     1  12.0
    -1     1    -1     1  21.0
     1    -1    -1     1  21.0
    -1    -1    -1     1  38.0
     1     1     1    -1   5.0
    -1     1     1    -1   0.0
     1    -1     1    -1   0.0
    -1    -1     1    -1   3.0
     1     1    -1    -1   0.0
    -1     1    -1    -1   3.0
     1    -1    -1    -1   3.0
    -1    -1    -1    -1  14.0


In [22]:
import dimod
import minorminer
from dwave.system.samplers import DWaveSampler 

my_solver = 'DW_2000Q_2'
my_token = 'DEV-6a2db0d6dd7d118b1bb0c458d9732dfe2f495cbd'

In [23]:
h = {1: -3, 2: -3, 3: -3, 4: 7.5}
J = {(1, 4): -1.5, (2, 4): -1.5, (3,4): -1.5, (1,2): 2, (1,3): 2, (2,3): 2}
bqm = dimod.BinaryQuadraticModel.from_ising(h, J, 11)

In [25]:
response = dimod.ExactSolver().sample(bqm)

row_format ="{:>6}" * 5
print(row_format.format('s1', 's2', 's3', 's4', 'E'))
for sample in response.data(fields=['sample', 'energy']):
    s = sample[0]
    print(row_format.format(s[1], s[2], s[3], s[4], sample[1]))

    s1    s2    s3    s4     E
     1     1    -1    -1   0.0
    -1     1     1    -1   0.0
     1    -1     1    -1   0.0
     1    -1    -1    -1   3.0
    -1     1    -1    -1   3.0
    -1    -1     1    -1   3.0
     1     1     1    -1   5.0
     1     1     1     1  11.0
     1    -1     1     1  12.0
    -1     1     1     1  12.0
     1     1    -1     1  12.0
    -1    -1    -1    -1  14.0
    -1    -1     1     1  21.0
    -1     1    -1     1  21.0
     1    -1    -1     1  21.0
    -1    -1    -1     1  38.0


In [26]:
sampler = DWaveSampler(solver=my_solver, token=my_token)
_, target_edgelist, target_adjacency = sampler.structure

embedding = minorminer.find_embedding(bqm.quadratic, target_edgelist)
if bqm and not embedding:
    raise ValueError("no embedding found")

bqm_embedded = dimod.embed_bqm(bqm, embedding, target_adjacency, 3.0)

print(len(list(embedding.keys())), "logical qubits")
phys = list()
for _, v in embedding.items():
    phys += v
print(len(phys), "physical qubits")

4 logical qubits
6 physical qubits


In [30]:
response = sampler.sample(bqm_embedded, num_reads=500, answer_mode='histogram')
response = dimod.unembed_response(response, embedding, source_bqm=bqm)
    
row_format ="{:>6}" * 6
print(row_format.format('s1', 's2', 's3', 's4', 'E', 'N'))
for sample in response.data(fields=['sample', 'energy', 'num_occurrences']):
    s = sample[0]
    print(row_format.format(s[1], s[2], s[3], s[4], sample[1], sample[2]))

    s1    s2    s3    s4     E     N
     1    -1     1    -1   0.0   155
    -1     1     1    -1   0.0   219
     1     1    -1    -1   0.0   125
    -1     1     1    -1   0.0     1


In [31]:
embedding

{1: [360], 2: [366], 3: [364, 363], 4: [365, 362]}